In [3]:
import torch
import adapters

from adapters import LlamaAdapterModel
from transformers import AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

In [6]:
model = LlamaAdapterModel.from_pretrained("meta-llama/Llama-2-7b-hf")

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.76s/it]


In [17]:
model.add_causal_lm_head("lm_head")

In [22]:
adapters.init(model)

LlamaAdapterModel(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayerWithAdapters(
        (self_attn): LlamaAttentionWithAdapters(
          (q_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (loras): ModuleDict()
          )
          (k_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (loras): ModuleDict()
          )
          (v_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (loras): ModuleDict()
          )
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
          (prefix_tuning): PrefixTuningShim(
            (prefix_gates): ModuleDict()
            (pool): PrefixTuningPool(
              (prefix_tunings): ModuleDict()
            )
          )
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_fe

In [15]:
[s for s in dir(model) if "head" in s]

['_active_heads',
 '_convert_head_mask_to_5d',
 '_convert_to_flex_head',
 '_copy_lm_head_original_to_resized',
 '_get_resized_lm_head',
 '_get_used_heads',
 '_init_head_modules',
 'active_head',
 'add_causal_lm_head',
 'add_classification_head',
 'add_custom_head',
 'add_prediction_head',
 'add_prediction_head_from_config',
 'delete_head',
 'forward_head',
 'get_head_mask',
 'get_prediction_heads_config',
 'head_types',
 'heads',
 'load_head',
 'prune_heads',
 'register_custom_head',
 'save_all_heads',
 'save_head']

In [23]:
out = model.generate(tokenizer("The cat is", return_tensors="pt")["input_ids"], max_length=20)

In [24]:
tokenizer.decode(out[0])

'<s> The cat isРСРoulevalu str siècleTest materadejkradeavenradebosebose étudesvä'

In [2]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b")
# model = T5ForConditionalGeneration.from_pretrained("t5-base")
quantization_config = None
# BitsAndBytesConfig(
    # load_in_4bit=True,
    # bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    # load_in_8bit=True,
# )

model = AutoModelForSeq2SeqLM.from_pretrained(
    "t5-base", quantization_config=quantization_config, torch_dtype=torch.bfloat16)
adapters.init(model)
model.add_adapter("adapter", config="pfeiffer", set_active=True)
model.train_adapter("adapter")

/mnt/shared_home/vlialin/miniconda3/envs/peft_comparison/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight', 'lm_he

RuntimeError: data set to a tensor that requires gradients must be floating point or complex dtype

In [30]:
input_ids = tokenizer("Earth rotates around <extra_id_0>", return_tensors="pt").input_ids
decoder_input_ids = tokenizer("<pad> <extra_id_0>", return_tensors="pt").input_ids

out = model.generate(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
tokenizer.decode(out[0])

'<pad><extra_id_0></s></s>'

In [ ]:
# model = T5AdapterModel.from_pretrained("t5-small")
# model.add_seq2seq_lm_head("lm_head")